


# First install Torch Geometric library


Ensure that you have Torch Geometric installed before proceeding.

In [1]:
! pip install  torch_geometric==2.4.0
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00
time: 381 µs (started: 2024-02-27 19:18:11 +00:00)


# `G2ACO`
**Import necessary libraries**  
Import the required Python libraries for your project.

In [2]:
from re import X
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score,adjusted_rand_score,accuracy_score
from scipy.sparse import coo_matrix
import numpy as np
from torch_geometric.data import Data
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F

time: 7.54 s (started: 2024-02-27 19:18:11 +00:00)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(50)
np.random.seed(50)

time: 74.3 ms (started: 2024-02-27 19:19:33 +00:00)


import datasets from torch geometric dataset

Load or import the dataset

In [4]:
# dataset name
dataset_name='pubmed'

# Load the dataset
root = '/content/'+dataset_name
dataset = Planetoid(root=root, name=dataset_name)


Processing...


time: 3.72 s (started: 2024-02-27 19:20:13 +00:00)


Done!


Define settings
Set up necessary configurations for G2ACO model.

In [5]:
num_communities=dataset.num_classes
num_nodes=dataset.data.num_nodes
y = dataset.data.y
dropout=0.7
dropout1=0.4

A = torch.tensor(dataset.edge_index, dtype=torch.long).clone().detach()
A = coo_matrix((np.ones(A.shape[1]), (A[0], A[1])), shape=(num_nodes, num_nodes))
A = torch.tensor(A.toarray(), dtype=torch.float).clone().detach()
data = Data(x=dataset.x, edge_index=dataset.edge_index, y=dataset.y)
# Convert to GPU if available
A=A.to(device)
data = data.to(device)

input_dim = data.x.shape[1]
hidden_dim = 32
latent_dim = 8
n_heads = 8

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
<ipython-input-5-bc85ef33ad99>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  A = torch.tensor(dataset.edge_index, dtype=torch.long).clone().detach()


time: 3.81 s (started: 2024-02-27 19:20:24 +00:00)


G2ACO model

In [6]:
torch.manual_seed(42)
np.random.seed(42)
class GVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, n_heads, dropout,dropout1):
        super(GVAE, self).__init__()
        torch.manual_seed(1234567)
        self.enc1 = GATConv(input_dim, hidden_dim, heads=n_heads)
        self.enc2 = GATConv(hidden_dim * n_heads, latent_dim, heads=n_heads)

    def encode(self, x, edge_index):
        x = nn.functional.dropout(x,p=dropout, training=self.training)
        x = self.enc1(x, edge_index)
        x = nn.functional.dropout(x,p=dropout1,  training=self.training) # 7 4
        x = self.enc2(x, edge_index)
        x  = F.normalize(x, p=2, dim=1)
        return x

    def decode(self, z):
        return nn.functional.sigmoid(torch.matmul(z, z.t()))

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        recon = self.decode(z)
        return recon, z


time: 5.56 ms (started: 2024-02-27 19:20:30 +00:00)


Define k-means clustering loss function
This loss function specifically designed for clustering tasks.

In [7]:
# Defin the k-means as a clustering loss function
def k_means_clustering_loss(embeddings, num_clusters):
     # Perform clustering using K-means
    kmeans = KMeans(n_clusters=num_clusters, max_iter=5000,n_init=20,init='k-means++')
    kmeans.fit(embeddings.detach().cpu().numpy())
    cluster_centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32, device=embeddings.device)
    # Compute pairwise distances between node embeddings and cluster centers
    pairwise_distances = torch.cdist(embeddings, cluster_centers)
    # Minimize the distance to the closest cluster center
    clustering_loss = torch.min(pairwise_distances, dim=1)[0].mean()
    return clustering_loss


time: 757 µs (started: 2024-02-27 19:20:36 +00:00)


Define optimizer

In [8]:
model = GVAE(input_dim, hidden_dim, latent_dim, n_heads, dropout,dropout1)
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.006, weight_decay=5e-3)
model = model.to(device)

time: 26.2 ms (started: 2024-02-27 19:20:38 +00:00)


Training loop and execution

In [9]:
# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    recon, z = model(data.x, data.edge_index)
    recon_loss = F.binary_cross_entropy(recon,A, reduction='mean')
    clustering_loss=k_means_clustering_loss(z,dataset.num_classes)
    loss = recon_loss + 10 *clustering_loss
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
for epoch in range(25):
    loss = train()
    print(f"Epoch: {epoch+1}, Loss: {loss:.4f}")


Epoch: 1, Loss: 9.7816
Epoch: 2, Loss: 4.3902
Epoch: 3, Loss: 3.0743
Epoch: 4, Loss: 2.5570
Epoch: 5, Loss: 2.2857
Epoch: 6, Loss: 2.1228
Epoch: 7, Loss: 2.0082
Epoch: 8, Loss: 1.9284
Epoch: 9, Loss: 1.8690
Epoch: 10, Loss: 1.8188
Epoch: 11, Loss: 1.7817
Epoch: 12, Loss: 1.7499
Epoch: 13, Loss: 1.7214
Epoch: 14, Loss: 1.6978
Epoch: 15, Loss: 1.6768
Epoch: 16, Loss: 1.6600
Epoch: 17, Loss: 1.6442
Epoch: 18, Loss: 1.6292
Epoch: 19, Loss: 1.6156
Epoch: 20, Loss: 1.6033
Epoch: 21, Loss: 1.5933
Epoch: 22, Loss: 1.5828
Epoch: 23, Loss: 1.5725
Epoch: 24, Loss: 1.5640
Epoch: 25, Loss: 1.5565
time: 1min 6s (started: 2024-02-27 19:20:43 +00:00)


Evaluation

In [10]:
model.eval()
with torch.no_grad():
     embeddings = model.encode(data.x, data.edge_index)
     embeddings = embeddings.cpu().detach().numpy()

# Perform clustering using K-means
kmeans = KMeans(n_clusters=dataset.num_classes,max_iter=5000, n_init=20,init='k-means++')
clusters = kmeans.fit_predict(embeddings)

    # Evaluate clustering performance using NMI and ARI
nmi = normalized_mutual_info_score((data.y).cpu(), clusters)
ari = adjusted_rand_score((data.y).cpu(), clusters)
print(f"NMI: {nmi:.4f}")
print(f"ARI: {ari:.4f}")

NMI: 0.3451
ARI: 0.3314
time: 1.38 s (started: 2024-02-27 19:22:59 +00:00)
